# Comparison between "Comprehensive upper-air observation network from 1905 to present" and "Insitu IGRA radiosoundings baseline network"

**Contains modified Copernicus Climate Change Service Information 2020**
under [License](https://apps.ecmwf.int/datasets/licences/copernicus/)

*Copernicus Climate Change Service (C3S) - Upper Air Data Service (2020)*

The purpose of this IPython Notebook is to find differences in both mentioned data sets and to compare them. 

    Author: U. Voggenberger
    Date: 02.2021
    Contact: ulrich.voggenberger [at] univie.ac.at
    License: C3S, 2021

In [24]:
import os, sys
euapath = os.getcwd()+'/../cds-backend/code/'
sys.path.append(euapath)
try:
    import cds_eua3 as eua
except:
    print('could not import eua')
    print('please download https://github.com/MBlaschek/CEUAS/blob/master/CEUAS/public/cds-backend/code/cds_eua3.py and save in ', euapath)
    print('or get the whole repository from https://github.com/MBlaschek/CEUAS')

In [25]:
import pandas
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
warnings.filterwarnings('ignore')


In [2]:
def request(rqdict, source, remove_file=True):
    
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        source,rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./download/')
    z.close()
    print('Request took: ' + str(time.time() - t0) + ' seconds')
    
    files = glob.glob('./download/*.nc')
    
    if files[0].split('/')[-1].startswith('IGRA'):
        ds = xarray.open_dataset(files[0])
        ds = ds.drop(['report_id'])
        data = ds.to_dataframe()
        for i in files[1:]:
            ds = xarray.open_dataset(i)            
            data = data.append(ds.to_dataframe())

    else:
        data=eua.CDMDataset(files[0]).to_dataframe()
        for i in files[1:]:
            da = eua.CDMDataset(i).to_dataframe()
            data = data.append(da)
            
    os.remove('download.zip')
    if remove_file:
        try:
            shutil.rmtree('./download/')
        except:
            print('Error while deleting directory')

    return data

---

In [3]:
cuon = 'insitu-comprehensive-upper-air-observation-network'
igra = 'insitu-observations-igra-baseline-network'

# short period - single station

In [7]:
t = '2000-01-01/2000-01-02'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [8]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:13:59,183 INFO Welcome to the CDS
2021-06-21 15:13:59,184 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-06-21 15:13:59,345 INFO Request is queued
2021-06-21 15:14:00,381 INFO Request is completed
2021-06-21 15:14:00,383 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.comprehensive_upper_air.retrieve-1624281239.5460596-13639-14-26186f48-85d1-4eaa-a94e-668be9632025.zip to download.zip (42.6K)
2021-06-21 15:14:00,497 INFO Download rate 375.5K/s
2021-06-21 15:14:00,570 INFO Reading Profile on slice(None, None, None)


Request took: 1.508411169052124 seconds


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.23,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.23,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.23,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.23,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
242,48.23,16.370001,94800.0,10000030641,WIEN/HOHE WARTE (11035-0),272.100006,2000-01-02 12:00:00,nnnnnnnnnnnnnn
243,48.23,16.370001,97800.0,10000030641,WIEN/HOHE WARTE (11035-0),273.600006,2000-01-02 12:00:00,nnnnnnnnnnnnnn
244,48.23,16.370001,98300.0,10000030641,WIEN/HOHE WARTE (11035-0),271.500000,2000-01-02 12:00:00,nnnnnnnnnnnnnn
245,48.23,16.370001,99100.0,10000030641,WIEN/HOHE WARTE (11035-0),271.100006,2000-01-02 12:00:00,nnnnnnnnnnnnnn


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.23,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.23,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.23,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.23,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
242,48.23,16.370001,94800.0,10000030641,WIEN/HOHE WARTE (11035-0),272.100006,2000-01-02 12:00:00,nnnnnnnnnnnnnn
243,48.23,16.370001,97800.0,10000030641,WIEN/HOHE WARTE (11035-0),273.600006,2000-01-02 12:00:00,nnnnnnnnnnnnnn
244,48.23,16.370001,98300.0,10000030641,WIEN/HOHE WARTE (11035-0),271.500000,2000-01-02 12:00:00,nnnnnnnnnnnnnn
245,48.23,16.370001,99100.0,10000030641,WIEN/HOHE WARTE (11035-0),271.100006,2000-01-02 12:00:00,nnnnnnnnnnnnnn


### IGRA

In [9]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:14:06,386 INFO Welcome to the CDS
2021-06-21 15:14:06,387 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-06-21 15:14:06,533 INFO Request is queued
2021-06-21 15:14:07,570 INFO Request is running
2021-06-21 15:14:09,106 INFO Request is completed
2021-06-21 15:14:09,107 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.insitu_reference.retrieve-1624281248.7213695-13759-14-2eda4f8b-70ee-4490-8bf9-b597586e2c38.zip to download.zip (2.8K)
2021-06-21 15:14:09,193 INFO Download rate 33K/s


Request took: 3.0882091522216797 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,7000,211.449997
1,AUM00011035,2000-01-02,16.3564,48.2486,92500,271.049988
2,AUM00011035,2000-01-02,16.3564,48.2486,70000,259.649994
3,AUM00011035,2000-01-02,16.3564,48.2486,5000,208.250000
4,AUM00011035,2000-01-02,16.3564,48.2486,3000,208.050003
...,...,...,...,...,...,...
81,AUM00011035,2000-01-02,16.3564,48.2486,92600,270.850006
82,AUM00011035,2000-01-02,16.3564,48.2486,40800,236.649994
83,AUM00011035,2000-01-02,16.3564,48.2486,6180,212.649994


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,7000,211.449997
1,AUM00011035,2000-01-02,16.3564,48.2486,92500,271.049988
2,AUM00011035,2000-01-02,16.3564,48.2486,70000,259.649994
3,AUM00011035,2000-01-02,16.3564,48.2486,5000,208.250000
4,AUM00011035,2000-01-02,16.3564,48.2486,3000,208.050003
5,AUM00011035,2000-01-02,16.3564,48.2486,2000,204.449997
6,AUM00011035,2000-01-02,16.3564,48.2486,30000,225.649994
7,AUM00011035,2000-01-02,16.3564,48.2486,85000,267.850006
8,AUM00011035,2000-01-02,16.3564,48.2486,15000,214.649994


# long period - single station

In [10]:
t = '2000-01-01/2020-01-01'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [11]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:14:28,444 INFO Welcome to the CDS
2021-06-21 15:14:28,445 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-06-21 15:14:28,604 INFO Request is queued
2021-06-21 15:14:29,640 INFO Request is running
2021-06-21 15:14:33,461 INFO Request is completed
2021-06-21 15:14:33,462 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1624281272.7088525-13912-14-02df1f11-e851-4639-8fcb-bf98d0fb2dff.zip to download.zip (85.5M)
2021-06-21 15:14:39,548 INFO Download rate 14.1M/s  
2021-06-21 15:14:40,459 INFO Reading Profile on slice(None, None, None)


Request took: 12.129385948181152 seconds


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.230000,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.230000,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.230000,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.230000,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
2423021,48.248951,16.356871,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423022,48.248951,16.356871,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423023,48.248951,16.356871,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423024,48.248951,16.356871,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,nnnnnnnnnnnnnn


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.230000,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.230000,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.230000,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.230000,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
2423021,48.248951,16.356871,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423022,48.248951,16.356871,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423023,48.248951,16.356871,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,nnnnnnnnnnnnnn
2423024,48.248951,16.356871,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,nnnnnnnnnnnnnn


### IGRA

In [12]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:14:49,546 INFO Welcome to the CDS
2021-06-21 15:14:49,548 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-06-21 15:14:49,697 INFO Request is queued
2021-06-21 15:14:50,732 INFO Request is running
2021-06-21 15:21:08,275 INFO Request is completed
2021-06-21 15:21:08,276 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve-1624281622.3839335-13709-14-31a6bb1a-8541-43d7-b71e-17d242ed2f4c.zip to download.zip (11.3M)
2021-06-21 15:21:09,547 INFO Download rate 8.9M/s   


Request took: 381.6020188331604 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,43200.0,239.250000
1,AUM00011035,2000-01-02,16.3564,48.2486,7000.0,211.449997
2,AUM00011035,2000-01-02,16.3564,48.2486,92500.0,271.049988
3,AUM00011035,2000-01-02,16.3564,48.2486,70000.0,259.649994
4,AUM00011035,2000-01-02,16.3564,48.2486,5000.0,208.250000
...,...,...,...,...,...,...
1392186,AUM00011035,2019-10-01,16.3564,48.2486,61700.0,270.850006
1392187,AUM00011035,2019-10-01,16.3564,48.2486,36900.0,246.449997
1392188,AUM00011035,2019-10-01,16.3564,48.2486,43600.0,255.050003


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,43200.0,239.250000
1,AUM00011035,2000-01-02,16.3564,48.2486,7000.0,211.449997
2,AUM00011035,2000-01-02,16.3564,48.2486,92500.0,271.049988
3,AUM00011035,2000-01-02,16.3564,48.2486,70000.0,259.649994
4,AUM00011035,2000-01-02,16.3564,48.2486,5000.0,208.250000
...,...,...,...,...,...,...
1392186,AUM00011035,2019-10-01,16.3564,48.2486,61700.0,270.850006
1392187,AUM00011035,2019-10-01,16.3564,48.2486,36900.0,246.449997
1392188,AUM00011035,2019-10-01,16.3564,48.2486,43600.0,255.050003


# short period - multiple stations

In [13]:
t = '2000-01-01/2000-02-01'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [14]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:21:12,780 INFO Welcome to the CDS
2021-06-21 15:21:12,781 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-06-21 15:21:12,945 INFO Request is queued
2021-06-21 15:21:13,982 INFO Request is running
2021-06-21 15:21:15,518 INFO Request is completed
2021-06-21 15:21:15,519 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data2/adaptor.comprehensive_upper_air.retrieve-1624281674.4302087-22759-15-603fb195-de57-44ed-bdcc-dca90abc883c.zip to download.zip (333.2K)
2021-06-21 15:21:15,779 INFO Download rate 1.3M/s
2021-06-21 15:21:15,890 INFO Reading Profile on slice(None, None, None)
2021-06-21 15:21:15,933 INFO Reading Profile on slice(None, None, None)
2021-06-21 15:21:15,963 INFO Reading Profile on slice(None, None, None)


Request took: 3.2533349990844727 seconds


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.23,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.23,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.23,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.23,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
3355,52.82,9.930000,80400.0,10000014511,BERGEN,270.700012,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3356,52.82,9.930000,85000.0,10000014511,BERGEN,272.899994,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3357,52.82,9.930000,85100.0,10000014511,BERGEN,272.899994,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3358,52.82,9.930000,92500.0,10000014511,BERGEN,276.799988,2000-02-01 23:00:00,nnnnnnnnnnnnnn


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.23,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.23,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.23,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.23,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
3355,52.82,9.930000,80400.0,10000014511,BERGEN,270.700012,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3356,52.82,9.930000,85000.0,10000014511,BERGEN,272.899994,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3357,52.82,9.930000,85100.0,10000014511,BERGEN,272.899994,2000-02-01 23:00:00,nnnnnnnnnnnnnn
3358,52.82,9.930000,92500.0,10000014511,BERGEN,276.799988,2000-02-01 23:00:00,nnnnnnnnnnnnnn


### IGRA

In [15]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
}, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:21:16,319 INFO Welcome to the CDS
2021-06-21 15:21:16,320 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-06-21 15:21:16,460 INFO Request is queued
2021-06-21 15:21:17,497 INFO Request is running
2021-06-21 15:21:21,321 INFO Request is completed
2021-06-21 15:21:21,322 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data1/adaptor.insitu_reference.retrieve-1624281680.0994222-23444-14-a6f02322-c6fa-4716-92e4-cc94102f50a0.zip to download.zip (178.7K)
2021-06-21 15:21:21,526 INFO Download rate 877.6K/s


Request took: 5.439506530761719 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,100000.0,279.750000
2,ITM00016245,2000-01-02,12.4508,41.669998,92500.0,275.350006
3,ITM00016245,2000-01-02,12.4508,41.669998,25000.0,218.449997
4,ITM00016245,2000-01-02,12.4508,41.669998,1000.0,210.850006
...,...,...,...,...,...,...
19705,AUM00011035,2000-02-01,16.3564,48.248600,2380.0,205.050003
19706,AUM00011035,2000-02-01,16.3564,48.248600,68500.0,267.450012
19707,AUM00011035,2000-02-01,16.3564,48.248600,13700.0,209.449997


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,100000.0,279.750000
2,ITM00016245,2000-01-02,12.4508,41.669998,92500.0,275.350006
3,ITM00016245,2000-01-02,12.4508,41.669998,25000.0,218.449997
4,ITM00016245,2000-01-02,12.4508,41.669998,1000.0,210.850006
...,...,...,...,...,...,...
19705,AUM00011035,2000-02-01,16.3564,48.248600,2380.0,205.050003
19706,AUM00011035,2000-02-01,16.3564,48.248600,68500.0,267.450012
19707,AUM00011035,2000-02-01,16.3564,48.248600,13700.0,209.449997


# long period - multiple stations

In [21]:
t = '2000-01-01/2020-01-02'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [22]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:48:35,034 INFO Welcome to the CDS
2021-06-21 15:48:35,035 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-06-21 15:48:35,196 INFO Request is queued
2021-06-21 15:48:36,231 INFO Request is running
2021-06-21 15:49:24,772 INFO Request is completed
2021-06-21 15:49:24,773 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data2/adaptor.comprehensive_upper_air.retrieve-1624283354.7401488-19624-12-201a1035-feb0-4186-a7ef-f6f4311ffcf2.zip to download.zip (451.5M)
2021-06-21 15:50:03,535 INFO Download rate 11.6M/s 
2021-06-21 15:50:07,909 INFO Reading Profile on slice(None, None, None)


Request took: 92.98380494117737 seconds


2021-06-21 15:50:22,217 INFO Reading Profile on slice(None, None, None)
2021-06-21 15:50:22,949 INFO Reading Profile on slice(None, None, None)


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.230000,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.230000,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.230000,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.230000,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
21826667,52.815201,9.924700,100660.0,10000043134,BERGEN,275.570007,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826668,52.815201,9.924700,100780.0,10000043134,BERGEN,275.640015,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826669,52.815201,9.924700,100900.0,10000043134,BERGEN,275.739990,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826670,52.815201,9.924700,101010.0,10000043134,BERGEN,275.779999,2020-01-02 22:46:02,nnnnnnnnnnnnnn


,lat,lon,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,nnnnnnnnnnnnnn
1,48.230000,16.370001,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
2,48.230000,16.370001,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
3,48.230000,16.370001,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,nnnnnnnnnnnnnn
4,48.230000,16.370001,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,nnnnnnnnnnnnnn
...,...,...,...,...,...,...,...,...
21826667,52.815201,9.924700,100660.0,10000043134,BERGEN,275.570007,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826668,52.815201,9.924700,100780.0,10000043134,BERGEN,275.640015,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826669,52.815201,9.924700,100900.0,10000043134,BERGEN,275.739990,2020-01-02 22:46:02,nnnnnnnnnnnnnn
21826670,52.815201,9.924700,101010.0,10000043134,BERGEN,275.779999,2020-01-02 22:46:02,nnnnnnnnnnnnnn


### IGRA 

In [23]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-06-21 15:51:49,282 INFO Welcome to the CDS
2021-06-21 15:51:49,284 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-06-21 15:51:49,490 INFO Request is queued
2021-06-21 15:51:50,525 INFO Request is running
2021-06-21 15:58:08,069 INFO Request is completed
2021-06-21 15:58:08,071 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve-1624283843.7603607-12784-14-5f759615-7e52-43db-82d3-e8ad530f74a1.zip to download.zip (46.8M)
2021-06-21 15:58:12,767 INFO Download rate 10M/s    


Request took: 387.79851150512695 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
2,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
3,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
4,ITM00016245,2000-01-02,12.4508,41.669998,10000.0,217.050003
...,...,...,...,...,...,...
5747116,GMM00010238,2020-01-02,9.9247,52.815300,101000.0,274.549988
5747117,GMM00010238,2020-01-02,9.9247,52.815300,83500.0,281.350006
5747118,GMM00010238,2020-01-02,9.9247,52.815300,58400.0,261.049988


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
2,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
3,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
4,ITM00016245,2000-01-02,12.4508,41.669998,10000.0,217.050003
...,...,...,...,...,...,...
5747116,GMM00010238,2020-01-02,9.9247,52.815300,101000.0,274.549988
5747117,GMM00010238,2020-01-02,9.9247,52.815300,83500.0,281.350006
5747118,GMM00010238,2020-01-02,9.9247,52.815300,58400.0,261.049988
